(There is no description for each code cell and what we want to accomplish, everything is in the lab1 pdf report (including the code), so it might be clearer.)

Professor: Guillaume Wisniewski

Class: Multilingual NLP

# Lab1: One model to rule them all?

One of the major issues in NLP is to find out if the deep learning models that achieve today state-of-the-art performances can be applied with the same success to all human languages. Indeed, these models have, in the vast majority of cases, only been tested on a very small number of languages generally qualified as 'resource rich', which have, from a linguistic point of view, many similarities. If it is therefore not surprising that, until now, a single model (i.e. the same neural network architecture) can be used to achieve very good performances, nothing prove that performance will not drop sharply when these models will be applied to a larger variety of languages.

This lab aims to provide a first answer to this question by verifying, in simple cases, whether there is a link between the performance of a model and certain characteristics of the languages to which it is applied.
In addition to studying this fascinating question (a personal opinion that is not debatable), this lab will also be an opportunity for you to learn how to train a very large number of models (you will have to train several models for each of the 41 languages considered in this lab!) and to highlight the need to automate your experiments.

In [ ]:
# install kenlm
!git clone https://github.com/kpu/kenlm.git
%cd kenlm
!python setup.py develop
!mkdir -p build
%cd build
!cmake ..
!make -j 4

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'kenlm': No such file or directory
[Errno 2] No such file or directory: 'kenlm'
/content/kenlm/build
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
python3: can't open file 'setup.py': [Errno 2] No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
mkdir: cannot create directory ‘build’: No such file or directory
[Errno 2] No such file or directory: 'build'
/content/kenlm/build
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Current working directory cannot be established.
shell-init: error retrievin

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
import tensorflow_datasets as tfds
from itertools import islice

## ds_fr = tfds.load("wiki40b/fr", split="test", data_dir="gs://tfds-data/datasets")
##sample = [ex["text"].numpy().decode("utf-8") for ex in islice(ds_fr.shuffle(buffer_size=10_000), 100)]

In [ ]:
!pip install pyicu
!pip install pycld2
!pip install morfessor
!pip install polyglot

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh

In [ ]:
# q13-15 (9min)
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import nltk
from polyglot.text import Text
from itertools import islice

# Define the languages
languages = [config.name for config in tfds.builder("wiki40b").builder_configs.values()]

# Iterate over the languages
for language in languages:
  # Load the corpus for the current language
  ds = tfds.load(f"wiki40b/{language}", split="test", data_dir="gs://tfds-data/datasets")

  # Preprocess the corpus
  sample = [ex["text"].numpy().decode("utf-8") for ex in islice(ds.shuffle(buffer_size=20_000), 3_000)]

  # Remove duplicates
  sample = list(set(sample))

  # Split into train and test sets
  train_sample, test_sample = train_test_split(sample, test_size=0.085, random_state=42)

  train_sample = ' '.join(train_sample)
  test_sample = ' '.join(test_sample)

  train_sample = Text(train_sample, hint_language_code = language)
  test_sample = Text(test_sample, hint_language_code = language)

  with open(f'{language}_train_tokenized.txt', 'w') as f:
    for sentence in train_sample.sentences:
      # Tokenize the sentence into words
      if sentence :
        words = sentence.words
        f.write(str(words) + ' ')
      # Write a newline character to separate sentences
      f.write('\n')

  with open(f'{language}_test_tokenized.txt', 'w') as f:
    for sentence in test_sample.sentences:
      # Tokenize the sentence into words
      if sentence :
        words = sentence.words
        f.write(str(words) + ' ')
      # Write a newline character to separate sentences
      f.write('\n')

print(f'done with {language}')


In [ ]:
# q16 : compute the TTR for each language

import json
from collections import Counter

# Fonction pour calculer le TTR
def compute_ttr(words):
  types = len(set(words))  # nombre de mots uniques
  tokens = len(words)      # nombre total de mots
  return types / tokens if tokens > 0 else 0

# Liste des langues (comme dans le code précédent)
languages = [config.name for config in tfds.builder("wiki40b").builder_configs.values()]

# Dictionnaire pour stocker les TTR
ttr_results = {}

# Calculer le TTR pour chaque dataset
for language in languages:
  for split in ['train', 'test']:
    tokenized_file = f'{language}_{split}_tokenized.txt'

    # Lire les mots du fichier
    with open(tokenized_file, 'r') as f:
      words = []
      for line in f:
        words_in_line = line.strip().split()  # Extraire les mots de la ligne
        words.extend(words_in_line)

    # Calculer le TTR
    ttr = compute_ttr(words)

    # Sauvegarder le TTR
    if language not in ttr_results:
      ttr_results[language] = {}
    ttr_results[language][split] = ttr

# Sauvegarder les résultats dans un fichier JSON
with open('ttr_results.json', 'w') as f:
  json.dump(ttr_results, f, indent=4)

In [ ]:
# q18
language_types = {
    'zh-cn': 'Isolating', 'zh-tw': 'Isolating', 'vi': 'Isolating', 'th': 'Isolating', 'id': 'Isolating',
    'en': 'Fusional', 'ar': 'Introflexive', 'nl': 'Fusional', 'fr': 'Fusional', 'de': 'Fusional', 'it': 'Fusional',
    'pl': 'Fusional', 'pt': 'Fusional', 'ru': 'Fusional', 'es': 'Fusional', 'bg': 'Fusional', 'ca': 'Fusional',
    'cs': 'Fusional', 'da': 'Fusional', 'el': 'Fusional', 'fa': 'Fusional', 'he': 'Introflexive', 'hi': 'Fusional',
    'hr': 'Fusional', 'hu': 'Fusional', 'lt': 'Fusional', 'lv': 'Fusional', 'no': 'Fusional', 'ro': 'Fusional',
    'sk': 'Fusional', 'sl': 'Fusional', 'sr': 'Fusional', 'sv': 'Fusional', 'uk': 'Fusional', 'ja': 'Agglutinative',
    'ko': 'Agglutinative', 'tr': 'Agglutinative', 'et': 'Agglutinative', 'fi': 'Agglutinative', 'tl': 'Agglutinative',
    'ms': 'Agglutinative'
}

In [ ]:
!pwd

/content/kenlm/build


In [ ]:
os.chdir('/content/kenlm/build')

In [ ]:
!pwd

/content/kenlm/build


In [ ]:
# q23
import subprocess

for language in languages:
  print(f'{language}')
  input_file = f'{language}_train_tokenized.txt'
  output_file = f'{language}_train.arpa'

  cmd = f"./bin/lmplz -o 5 < {input_file} > {output_file}"

  subprocess.run(cmd, shell=True, check=True)

  print(f'Language model for {language} has been saved to {output_file}')


en


CalledProcessError: Command './bin/lmplz -o 5 < en_train_tokenized.txt > en_train.arpa' returned non-zero exit status 2.

In [ ]:
import os
print(os.getcwd())

/content/kenlm/build


In [ ]:
# q24 : compute the perplexity

import kenlm

dico_ppl = {}
for language in languages:
  m = kenlm.Model(f'{language}_train.arpa')
  L_ppl = [] # pour stocker les perplexités pour un langage
  with open('{language}_train_tokenized.txt', 'r') as f:
    for sentence in f: # une ligne = une phrase
      sentence = sentence.strip() # enlève les espaces en début et fin de chaque phrase. Cela évite les erreurs de calcul de perplexité sur des lignes vides ou contenant uniquement des espaces.
      if sentence: # ignorer les lignes vides
        ppl = m.perplexity(sentence)
        L_ppl.append(ppl)

  # calculer la moy des ppl pour cette langue
  avg_perplexity = sum(L_ppl) / len(L_ppl) if L_ppl else float('inf')

  # Stocker la perplexité moyenne dans le dictionnaire
  dico_ppl[language] = avg_perplexity

print(dico_ppl)

AttributeError: module 'kenlm' has no attribute 'Model'

# Brouillon

In [ ]:
# ds = tfds.load("wiki40b", split="train", data_dir="gs://tfds-data/datasets") #j'ai mis TRAIN # pas nécessaire

languages = [config.name for config in tfds.builder("wiki40b").builder_configs.values()]
print(languages)
for language in languages:
  print(language)
  ds = tfds.load(f"wiki40b/{language}", split="train", data_dir="gs://tfds-data/datasets")
  sample = [ex["text"].numpy().decode("utf-8") for ex in islice(ds.shuffle(buffer_size=10_000), 100)] # liste de 100 articles


In [ ]:
# PROGRAMME ORIGINEL (q13)

from polyglot.text import Text

# Obtenir la liste des langues disponibles dans Wiki40B
languages = [config.name for config in tfds.builder("wiki40b").builder_configs.values()]
print(languages)

# Fonction pour segmenter un texte en phrases uniques
def extract_unique_sentences(text):
  pg_text = Text(text)
  sentences = [str(sentence) for sentence in pg_text.sentences]  # Convertir en chaînes de caractères
  # avant : sentences = list(pg_text.sentences)  # Segmenter en phrases
  return list(set(sentences))  # Supprimer les doublons

# Fonction pour charger les articles et extraire des phrases
def load_and_extract_sentences(lang_code, split, num_sentences, num_articles=100):
  ds = tfds.load(f"wiki40b/{lang_code}", split=split, data_dir="gs://tfds-data/datasets")
  sample = [ex["text"].numpy().decode("utf-8") for ex in islice(ds.shuffle(buffer_size=10_000), num_articles)]

  # Extraire les phrases uniques des articles
  sentences = []
  for article in sample:
    sentences.extend(extract_unique_sentences(article))

  return list(set(sentences))[:num_sentences]  # Retourner un ensemble de phrases uniques

for lang in languages: # 2000 sentences for train set
  print(lang)
  L_train = load_and_extract_sentences(lang_code=lang, split='train', num_sentences = 2000, num_articles=100)
for lang in languages: # 300 sentences for test set
  L_test = load_and_extract_sentences(lang_code=lang, split='test', num_sentences = 300, num_articles=100)

print(L_train[:3])

['en', 'ar', 'zh-cn', 'zh-tw', 'nl', 'fr', 'de', 'it', 'ja', 'ko', 'pl', 'pt', 'ru', 'es', 'th', 'tr', 'bg', 'ca', 'cs', 'da', 'el', 'et', 'fa', 'fi', 'he', 'hi', 'hr', 'hu', 'id', 'lt', 'lv', 'ms', 'no', 'ro', 'sk', 'sl', 'sr', 'sv', 'tl', 'uk', 'vi']
en
ar


zh-cn


KeyboardInterrupt: 

In [ ]:
datasets = tfds.list_builders()

# Afficher tous les jeux de données pour vérification
print("Jeux de données disponibles dans TensorFlow Datasets :")
for dataset in datasets:
    print(dataset)

Jeux de données disponibles dans TensorFlow Datasets :
abstract_reasoning
accentdb
aeslc
aflw2k3d
ag_news_subset
ai2_arc
ai2_arc_with_ir
aloha_mobile
amazon_us_reviews
anli
answer_equivalence
arc
asqa
asset
assin2
asu_table_top_converted_externally_to_rlds
austin_buds_dataset_converted_externally_to_rlds
austin_sailor_dataset_converted_externally_to_rlds
austin_sirius_dataset_converted_externally_to_rlds
bair_robot_pushing_small
bc_z
bccd
beans
bee_dataset
beir
berkeley_autolab_ur5
berkeley_cable_routing
berkeley_fanuc_manipulation
berkeley_gnm_cory_hall
berkeley_gnm_recon
berkeley_gnm_sac_son
berkeley_mvp_converted_externally_to_rlds
berkeley_rpt_converted_externally_to_rlds
big_patent
bigearthnet
billsum
binarized_mnist
binary_alpha_digits
ble_wind_field
blimp
booksum
bool_q
bot_adversarial_dialogue
bridge
bridge_data_msr
bucc
c4
c4_wsrs
caltech101
caltech_birds2010
caltech_birds2011
cardiotox
cars196
cassava
cats_vs_dogs
celeb_a
celeb_a_hq
cfq
cherry_blossoms
chexpert
cifar10
cifar1